#*Install & Download Package**

In [ ]:
!pip install tweet-preprocessor
!pip install textblob
!pip install sastrawi
!pip install PySastrawi
!pip install swifter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=3a5eda2220292393ecfcb656cc393b8b9c5f26b3b267ce5628ddb689eec99211
  Stored in directory: /root/.cache/pip/wheels/d9/31/ff/ff51141a088571a9f672449e5aad5ea8bb35ca5d95ba135f30
Successfully built swifter


In [ ]:
import pandas as pd
import re
import string
import preprocessor as p
import nltk
nltk.download('stopwords')
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
from preprocessor.api import clean, tokenize, parse
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import wordcloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#**Preprocessing**

In [ ]:
tweets_df = pd.read_csv('/content/cleanDataFINAL.csv')
tweets_df

,Clean Tweet
0,koalisi masyarakat sipil peduli bencana nilai ...
1,cc dasco negara berani sebut bencana nasional ...
2,banda aceh gubernur aceh muzakir manaf mualem ...
3,orang aceh banjir bandang parah tsunami pas pa...
4,jembatan darurat bangun warga reje payung ling...
...,...
1067,bdx indonesia indosat kuat respons darurat ban...
1068,mensos bansos korban banjir sumatera bagi januari
1069,kemenpu adhi tangan sungai dampak banjir sumat...
1070,perintah tni optimal bersih sekolah pesantren ...


In [ ]:
#case folding
tweets_df['Case Folding'] = tweets_df['Clean Tweet'].str.lower()
tweets_df

,Clean Tweet,Case Folding
0,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...
1,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...
2,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...
3,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...
4,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...
...,...,...
1067,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...
1068,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari
1069,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...
1070,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...


In [ ]:
#data cleansing
#remove special char

def remove_tweet_special(text):
    #remove tab, new line, and back slice
    text = str(text).replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"").replace('\n',"").replace('ud83d',"")
    #remove non ASCII
    text = text.encode('ascii', 'replace').decode('ascii')
    text = re.sub(r'^rt[\s]+', '',text)
    #remove mention, link, hastag
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    #remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
tweets_df['Data Cleansing']= tweets_df['Case Folding'].apply(remove_tweet_special)
#remove number
def remove_number(text):
    return re.sub(r"\d+", "", text)
tweets_df['Data Cleansing']= tweets_df['Data Cleansing'].apply(remove_number)
#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))
tweets_df['Data Cleansing']= tweets_df['Data Cleansing'].apply(remove_punctuation)
#remove whitespace
def remove_whitespace_LT(text):
    return text.strip()
tweets_df['Data Cleansing']= tweets_df['Data Cleansing'].apply(remove_whitespace_LT)

#remove multiple whitespace
def remove_whitespace_multiple(text):
    return re.sub(']s+',' ',text)
tweets_df['Data Cleansing']= tweets_df['Data Cleansing'].apply(remove_whitespace_multiple)

#remove user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt
tweets_df['Data Cleansing']= np.vectorize(remove_pattern)(tweets_df['Data Cleansing'], "@[\w]*")

#remove single char
def remove_singel_char(text):
    return  re.sub(r"\b[a-zA-Z]\b", "", text)
tweets_df['Data Cleansing']= tweets_df['Data Cleansing'].apply(remove_singel_char)
tweets_df.head()

,Clean Tweet,Case Folding,Data Cleansing
0,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...
1,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...
2,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...
3,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...
4,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...


In [ ]:
#remove duplicate
tweets_df = tweets_df.drop_duplicates(subset=["Data Cleansing"])
tweets_df

,Clean Tweet,Case Folding,Data Cleansing
0,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...
1,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...
2,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...
3,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...
4,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...
...,...,...,...
1067,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...
1068,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari
1069,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...
1070,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...


In [ ]:
#Tokenization
def tokenization(text):
    text= re.split('\W+', text)
    return text
tweets_df['Tokenisasi']= tweets_df['Data Cleansing'].apply(tokenization)
tweets_df

,Clean Tweet,Case Folding,Data Cleansing,Tokenisasi
0,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,"[koalisi, masyarakat, sipil, peduli, bencana, ..."
1,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,"[cc, dasco, negara, berani, sebut, bencana, na..."
2,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,"[banda, aceh, gubernur, aceh, muzakir, manaf, ..."
3,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,"[orang, aceh, banjir, bandang, parah, tsunami,..."
4,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,"[jembatan, darurat, bangun, warga, reje, payun..."
...,...,...,...,...
1067,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,"[bdx, indonesia, indosat, kuat, respons, darur..."
1068,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,"[mensos, bansos, korban, banjir, sumatera, bag..."
1069,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,"[kemenpu, adhi, tangan, sungai, dampak, banjir..."
1070,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,"[perintah, tni, optimal, bersih, sekolah, pesa..."


In [ ]:
#normalisasi
normalized_word = pd.read_csv("normalisasi.csv")
normalized_word_dict = {}
for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]
tweets_df['Normalisasi'] = tweets_df['Tokenisasi'].apply(normalized_term)
tweets_df

,Clean Tweet,Case Folding,Data Cleansing,Tokenisasi,Normalisasi
0,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,"[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ..."
1,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,"[cc, dasco, negara, berani, sebut, bencana, na...","[cc, dasco, negara, berani, sebut, bencana, na..."
2,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,"[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ..."
3,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,"[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,..."
4,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,"[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun..."
...,...,...,...,...,...
1067,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,"[bdx, indonesia, indosat, kuat, respons, darur...","[bdx, indonesia, indosat, kuat, respons, darur..."
1068,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,"[mensos, bansos, korban, banjir, sumatera, bag...","[mensos, bansos, korban, banjir, sumatera, bag..."
1069,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,"[kemenpu, adhi, tangan, sungai, dampak, banjir...","[kemenpu, adhi, tangan, sungai, dampak, banjir..."
1070,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,"[perintah, tni, optimal, bersih, sekolah, pesa...","[perintah, tni, optimal, bersih, sekolah, pesa..."


In [ ]:
#Stopwords Removal
stopword = nltk.corpus.stopwords.words('indonesian')
stopword.extend(["yg","dg","rt","dgn","ny","d",'klo','kalo','amp',
                 'biar','bikin', 'bilang','gak', 'ga', 'krn', 'nya',
                 'nih', 'sih','si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                 'jd', 'jgn', 'sdh', 'aja', 'n', 't','nyg', 'hehe', 'pen',
                 'u', 'nan', 'loh','rt','&amp', 'yah','mah'])

txt_stopword = pd.read_csv("stopword.csv", names=["stopwords"], header = None)
stopword.extend(txt_stopword["stopwords"][0].split(' '))
stopword = set(stopword)
def stopwords(text):
    text= [word for word in text if word not in stopword]
    return text
tweets_df['Stopwords']= tweets_df["Normalisasi"].apply(stopwords)
tweets_df

,Clean Tweet,Case Folding,Data Cleansing,Tokenisasi,Normalisasi,Stopwords
0,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,"[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ..."
1,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,"[cc, dasco, negara, berani, sebut, bencana, na...","[cc, dasco, negara, berani, sebut, bencana, na...","[cc, dasco, negara, berani, bencana, nasional,..."
2,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,"[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ..."
3,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,"[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,..."
4,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,"[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun..."
...,...,...,...,...,...,...
1067,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,"[bdx, indonesia, indosat, kuat, respons, darur...","[bdx, indonesia, indosat, kuat, respons, darur...","[bdx, indonesia, indosat, kuat, respons, darur..."
1068,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,"[mensos, bansos, korban, banjir, sumatera, bag...","[mensos, bansos, korban, banjir, sumatera, bag...","[mensos, bansos, korban, banjir, sumatera, jan..."
1069,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,"[kemenpu, adhi, tangan, sungai, dampak, banjir...","[kemenpu, adhi, tangan, sungai, dampak, banjir...","[kemenpu, adhi, tangan, sungai, dampak, banjir..."
1070,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,"[perintah, tni, optimal, bersih, sekolah, pesa...","[perintah, tni, optimal, bersih, sekolah, pesa...","[perintah, tni, optimal, bersih, sekolah, pesa..."


In [ ]:
import swifter
#create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
#stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)
term_dict = {}
for document in tweets_df['Stopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
print(len(term_dict))
print("------------------------")
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
print(term_dict)
print("------------------------")
#apply stemmed term to datefrom
def get_stemmed_term(document):
    return [term_dict[term] for term in document]
tweets_df['Stemmer'] = tweets_df['Stopwords'].apply(get_stemmed_term)
print(tweets_df['Stemmer'])

4568
------------------------
koalisi : koalisi
masyarakat : masyarakat
sipil : sipil
peduli : peduli
bencana : bencana
nilai : nilai
status : status
darurat : darurat
aceh : aceh
bukti : bukti
gagal : gagal
negara : negara
tangan : tangan
banjir : banjir
longsor : longsor
sumatra : sumatra
november : november
kritik : kritik
perintah : perintah
pusat : pusat
enggan : enggan
nasional : nasional
cc : cc
dasco : dasco
berani : berani
peter : peter
gontha : gontha
eks : eks
dubes : dubes
ri : ri
polandia : polandia
sumatera : sumatera
utara : utara
barat : barat
landa : landa
korban : korban
jiwa : jiwa
jatuh : jatuh
rumah : rumah
hancur : hancur
alam : alam
rusak : rusak
pilih : pilih
diam : diam
banda : banda
gubernur : gubernur
muzakir : muzakir
manaf : manaf
mualem : mualem
tanggap : tanggap
hidrometeorologi : hidrometeorologi
kali : kali
tujuh : tujuh
hitung : hitung
januari : januari
putus : putus
ambil : ambil
dasar : dasar
hasil : hasil
koordinasi : koordinasi
orang : orang
bandan

In [ ]:
tweets_df

,Clean Tweet,Case Folding,Data Cleansing,Tokenisasi,Normalisasi,Stopwords,Stemmer
0,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,"[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ..."
1,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,"[cc, dasco, negara, berani, sebut, bencana, na...","[cc, dasco, negara, berani, sebut, bencana, na...","[cc, dasco, negara, berani, bencana, nasional,...","[cc, dasco, negara, berani, bencana, nasional,..."
2,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,"[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ..."
3,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,"[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,..."
4,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,"[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun..."
...,...,...,...,...,...,...,...
1067,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,bdx indonesia indosat kuat respons darurat ban...,"[bdx, indonesia, indosat, kuat, respons, darur...","[bdx, indonesia, indosat, kuat, respons, darur...","[bdx, indonesia, indosat, kuat, respons, darur...","[bdx, indonesia, indosat, kuat, respons, darur..."
1068,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,mensos bansos korban banjir sumatera bagi januari,"[mensos, bansos, korban, banjir, sumatera, bag...","[mensos, bansos, korban, banjir, sumatera, bag...","[mensos, bansos, korban, banjir, sumatera, jan...","[mensos, bansos, korban, banjir, sumatera, jan..."
1069,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,kemenpu adhi tangan sungai dampak banjir sumat...,"[kemenpu, adhi, tangan, sungai, dampak, banjir...","[kemenpu, adhi, tangan, sungai, dampak, banjir...","[kemenpu, adhi, tangan, sungai, dampak, banjir...","[kemenpu, adhi, tangan, sungai, dampak, banjir..."
1070,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,perintah tni optimal bersih sekolah pesantren ...,"[perintah, tni, optimal, bersih, sekolah, pesa...","[perintah, tni, optimal, bersih, sekolah, pesa...","[perintah, tni, optimal, bersih, sekolah, pesa...","[perintah, tni, optimal, bersih, sekolah, pesa..."


In [ ]:
#remove punctuation
def remove_punct(text):
  text = " ".join([char for char in text if char not in string.punctuation])
  return text
tweets_df['Clean Tweet'] = tweets_df['Stemmer'].apply(lambda x: remove_punct(x))
tweets_df.head(11)

,Clean Tweet,Case Folding,Data Cleansing,Tokenisasi,Normalisasi,Stopwords,Stemmer
0,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,koalisi masyarakat sipil peduli bencana nilai ...,"[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ...","[koalisi, masyarakat, sipil, peduli, bencana, ..."
1,cc dasco negara berani bencana nasional peter ...,cc dasco negara berani sebut bencana nasional ...,cc dasco negara berani sebut bencana nasional ...,"[cc, dasco, negara, berani, sebut, bencana, na...","[cc, dasco, negara, berani, sebut, bencana, na...","[cc, dasco, negara, berani, bencana, nasional,...","[cc, dasco, negara, berani, bencana, nasional,..."
2,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,banda aceh gubernur aceh muzakir manaf mualem ...,"[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ...","[banda, aceh, gubernur, aceh, muzakir, manaf, ..."
3,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,orang aceh banjir bandang parah tsunami pas pa...,"[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,...","[orang, aceh, banjir, bandang, parah, tsunami,..."
4,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,jembatan darurat bangun warga reje payung ling...,"[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun...","[jembatan, darurat, bangun, warga, reje, payun..."
5,bencana nasional,tetap bencana nasional,tetap bencana nasional,"[tetap, bencana, nasional]","[tetap, bencana, nasional]","[bencana, nasional]","[bencana, nasional]"
6,masyarakat tuntut negara bencana nasional tent...,masyarakat tuntut negara tetap bencana nasiona...,masyarakat tuntut negara tetap bencana nasiona...,"[masyarakat, tuntut, negara, tetap, bencana, n...","[masyarakat, tuntut, negara, tetap, bencana, n...","[masyarakat, tuntut, negara, bencana, nasional...","[masyarakat, tuntut, negara, bencana, nasional..."
7,data badan nasional tanggulang bencana bnpb la...,data badan nasional tanggulang bencana bnpb la...,data badan nasional tanggulang bencana bnpb la...,"[data, badan, nasional, tanggulang, bencana, b...","[data, badan, nasional, tanggulang, bencana, b...","[data, badan, nasional, tanggulang, bencana, b...","[data, badan, nasional, tanggulang, bencana, b..."
8,bencana nasional ngak masuk bantu internasiona...,bencana nasional ngak masuk bantu internasiona...,bencana nasional ngak masuk bantu internasiona...,"[bencana, nasional, ngak, masuk, bantu, intern...","[bencana, nasional, ngak, masuk, bantu, intern...","[bencana, nasional, ngak, masuk, bantu, intern...","[bencana, nasional, ngak, masuk, bantu, intern..."
9,masyarakat aceh paksa angkut barang dagang but...,masyarakat aceh paksa angkut barang dagang but...,masyarakat aceh paksa angkut barang dagang but...,"[masyarakat, aceh, paksa, angkut, barang, daga...","[masyarakat, aceh, paksa, angkut, barang, daga...","[masyarakat, aceh, paksa, angkut, barang, daga...","[masyarakat, aceh, paksa, angkut, barang, daga..."


In [ ]:
# Install library yang diperlukan
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Membaca file CSV
#df = pd.read_csv('/content/cleanDataFINAL_translate (1).csv')

# Menampilkan beberapa baris pertama dari file untuk melihat struktur data
#df.head()

# Memilih kolom yang berisi teks yang ingin dihitung TF-IDF-nya
documents = tweets_df['Stemmer'].astype(str).tolist()

# Membuat objek TfidfVectorizer tanpa stop words dan pengaturan min_df
vectorizer = TfidfVectorizer(stop_words=None, max_df=1.0, min_df=1)

# Menghitung TF-IDF untuk kalimat
tfidf_matrix = vectorizer.fit_transform(documents)

# Menyusun dataframe untuk melihat hasilnya
tfidf_df = pd.DataFrame(tfidf_matrix.T.toarray(), index=vectorizer.get_feature_names_out(), columns=[f"Dokumen {i+1}" for i in range(len(documents))])

# Menampilkan tabel hasil TF-IDF
tfidf_df.head()  # Menampilkan hanya beberapa baris pertama untuk pemeriksaan

,Dokumen 1,Dokumen 2,Dokumen 3,Dokumen 4,Dokumen 5,Dokumen 6,Dokumen 7,Dokumen 8,Dokumen 9,Dokumen 10,...,Dokumen 1060,Dokumen 1061,Dokumen 1062,Dokumen 1063,Dokumen 1064,Dokumen 1065,Dokumen 1066,Dokumen 1067,Dokumen 1068,Dokumen 1069
ababil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abang,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abangan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abdi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#remove kolom
tweets_df.drop(tweets_df.columns[[0,1,2,3,4,5,6]], axis=1, inplace=True)
tweets_df

,Clean Tweet
0,putus ugal ugalan dpr perintah sebab orang ora...
1,video analisis bayar trilyun boardofpeace ayo ...
2,breaking perintah as shutdown jam aktivitas pe...
3,suara orang orang dizholimi perintah
4,warga surabaya hak tolak bayar parkir bahan ba...
...,...
136,perintah cepat demutualisasi bei jadi usaha bu...
137,happy banget tweet benci perintah benci jokowi...
272,chinese super league poin klub klub negatif ka...
273,perintah perhati gaji guru niat cerdas generas...


In [ ]:
#simpan data clean
tweets_df.to_csv('cleanData.csv', encoding='utf-8', index=False)

In [ ]:
#install dan import lib google translate
!pip install googletrans==3.1.0a0
import googletrans
from googletrans import Translator

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=1707a0abdb8da2e841d3baa2b62e6ab142522d5a56d574c8ab3be4fd5fdd9764
  Stored in directory: /root/.cache/pip/wheels/96/ac/bd/9df9eab356c0576896e97147425987f6f45e9e46456c978d18
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperframe

In [ ]:
df = pd.read_csv('cleanData.csv')
df

,Clean Tweet
0,koalisi masyarakat sipil peduli bencana nilai ...
1,banda aceh gubernur aceh muzakir manaf mualem ...
2,tetap bencana nasional paksa audit besar sebab...
3,rakyat aceh kesal bencana nasional banjir band...
4,breaking news mualem tetap status transisi dar...
...,...
218,bencana nasional mata ayo gerak lawan
219,gempa jogja erupsi rapi kiamat gunung kelud co...
220,guru honorer angkat angkat honor tigaratus reb...
221,orang tinggal akibat banjir sumatera pimpin di...


In [ ]:
#translate ke bahasa inggris
translator = Translator()
translations = {}
for column in df.columns:
  unique_elements = df[column].unique()
  for element in unique_elements:
    translations[element] = translator.translate(element).text
df.replace(translations, inplace=True)
df

,Clean Tweet
0,coalition of civil society concerned about dis...
1,Banda Aceh Aceh Governor Muzakir Manaf Mualem ...
2,"Still, national disasters force major audits d..."
3,The people of Aceh are upset about the nationa...
4,breaking news nauseam remains in emergency tra...
...,...
218,national disaster eye let's move the opponent
219,"Jogja earthquake erupts neatly, Mount Kelud Co..."
220,honorary teachers take up three hundred thousa...
221,people are living as a result of the Sumatran ...


In [ ]:
#case folding
df['Clean Tweet'] = df['Clean Tweet'].str.lower()
df

,Clean Tweet
0,coalition of civil society concerned about dis...
1,banda aceh aceh governor muzakir manaf mualem ...
2,"still, national disasters force major audits d..."
3,the people of aceh are upset about the nationa...
4,breaking news nauseam remains in emergency tra...
...,...
218,national disaster eye let's move the opponent
219,"jogja earthquake erupts neatly, mount kelud co..."
220,honorary teachers take up three hundred thousa...
221,people are living as a result of the sumatran ...


In [ ]:
#simpan data translate
df.to_csv('cleanData_translate1.csv', encoding='utf-8', index=False)